In [19]:
#課題2
# rmode.py: 丸めモード変更
# [Windows&Linux]
#  https://rafaelbarreto.wordpress.com/2009/03/30/controlling-fpu-rounding-modes-with-python/
# [macOS]
#  https://stackoverflow.com/questions/16000574/tie-breaking-of-round-with-numpy

# OS種別
# https://stackoverflow.com/questions/1854/python-what-os-am-i-running-on
import platform

# DLL呼び出し
# from ctypes import cdll
from ctypes import *

# 丸めモード定義
global FE_TOZERO, FE_DOWNWARD, FE_UPWARD, FE_TONEAREST
# set_rmode(丸めモード), 丸めモード = get_rmode()
global set_rmode, get_rmode

# OS名
os_name = platform.system()

# Windows
if os_name == 'Windows':
    msvcrt = cdll.msvcrt
    MW_RC = 0x00000300

    def set_rmode(mode):
        msvcrt._controlfp_s(0, mode, MW_RC)

    FE_TOZERO = 0x00000300
    FE_DOWNWARD = 0x00000100
    FE_UPWARD = 0x00000200
    FE_TONEAREST = 0x00000000

    def get_rmode():
        current_rmode = c_uint()
        msvcrt._controlfp_s(byref(current_rmode), 0, 0)

        return current_rmode.value & MW_RC

# Linux
elif os_name == 'Linux':
    from ctypes.util import find_library
    libm = cdll.LoadLibrary(find_library('m'))
    set_rmode, get_rmode = libm.fesetround, libm.fegetround
    # x86
    FE_TOZERO = 0xc00
    FE_DOWNWARD = 0x400
    FE_UPWARD = 0x800
    FE_TONEAREST = 0
# macOS
elif os_name == "Darwin":
    from ctypes.util import find_library
    libc = cdll.LoadLibrary(find_library('c'))
    set_rmode, get_rmode = libc.fesetround, libc.fegetround
    # x86
    FE_TOZERO = 0xc00
    FE_DOWNWARD = 0x400
    FE_UPWARD = 0x800
    FE_TONEAREST = 0


# 丸めモード確認
def print_rmode():
    rmode = get_rmode()

    if rmode == FE_TOZERO:
        print('切り捨て')
    elif rmode == FE_DOWNWARD:
        print('-Infへの丸め')
    elif rmode == FE_UPWARD:
        print('+Infへの丸め')
    elif rmode == FE_TONEAREST:
        print('最近偶数値丸め')
    else:
        print('不明 ', rmode)

    return rmode


# -------------------------------------
# Copyright (c) 2021 Tomonori Kouya
# All rights reserved.
# -------------------------------------

print_rmode()
x_rn=[0.7501]

for i in range(0,100):
    x_rn.append(4*x_rn[i]*(1-x_rn[i]))
    
set_rmode(FE_DOWNWARD)
print_rmode()
x_rm=[0.7501]

for i in range(0,100):
    x_rm.append(4*x_rm[i]*(1-x_rm[i]))
    
set_rmode(FE_UPWARD)
print_rmode()
x_rp=[0.7501]

for i in range(0,100):
    x_rp.append(4*x_rp[i]*(1-x_rp[i]))
    
rel_diff_rn_rm=[abs((x_rn[i]-x_rm[i])/x_rn[i])for i in range(len(x_rn))]    
rel_diff_rn_rp=[abs((x_rn[i]-x_rp[i])/x_rn[i])for i in range(len(x_rn))]    
rel_diff_rm_rp=[abs((x_rm[i]-x_rp[i])/x_rn[i])for i in range(len(x_rn))]
max_rel_diff=[
    max(
    rel_diff_rn_rm[i],
    rel_diff_rn_rp[i],
    rel_diff_rm_rp[i]
    )for i in range(len(x_rn))
]

print('    i,         x_rm[i]         ,         x_rn[i]         ,         x_rp[i]         ,max_rel_diff')
for i in range(0,101):
    if i%10==0:
        print(f'{i:5d},{x_rm[i]:25.17e},{x_rn[i]:25.17e},{x_rp[i]:25.17e},{max_rel_diff[i]:5.1e}')

+Infへの丸め
-Infへの丸め
+Infへの丸め
    i,         x_rm[i]         ,         x_rn[i]         ,         x_rp[i]         ,max_rel_diff
    0,  7.50099999999999989e-01,  7.50099999999999989e-01,  7.50099999999999989e-01,0.0e+00
   10,  8.44495953602235394e-01,  8.44495953602203309e-01,  8.44495953602203309e-01,3.8e-14
   20,  1.42939724494728609e-01,  1.42939724526234518e-01,  1.42939724526234518e-01,2.2e-10
   30,  8.54295985559328397e-01,  8.54296018080507924e-01,  8.54296018080507924e-01,3.8e-08
   40,  7.74953760069827080e-01,  7.74993177338575201e-01,  7.74993177338575201e-01,5.1e-05
   50,  1.09649817246645534e-01,  8.13185242954565651e-02,  8.13185242954565651e-02,3.5e-01
   60,  9.11998235902822794e-04,  5.22706152743196872e-01,  5.22706152743196872e-01,1.0e+00
   70,  2.19111632472834106e-01,  9.11054520908380128e-01,  9.11054520908380128e-01,7.6e-01
   80,  4.05026607676156281e-01,  6.17512393736452836e-01,  6.17512393736452836e-01,3.4e-01
   90,  1.95417967774143220e-01,  7.551369172230